In [ ]:
import gym
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate
import matplotlib.pyplot as plt
from collections import OrderedDict


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
#wrapper for cont env with PPO, plot result for several steps
def ppo_eval_interval(p, L, t_t, n_iter, n_step, gae, learning_rate, bias):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1, 'action':20}
    #PolicyCONFIG = dict(activation_fn=th.nn.Sigmoid,
                    # net_arch=[dict(pi=[16, 16], vf=[64,64,64])])
    cont_env = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=0, gamma = 1, gae_lambda=gae,
                     learning_rate = learning_rate,use_sde = False, n_steps = n_step)
    cont_model.set_parameters({'policy': OrderedDict([
                         ('action_net.weight', th.zeros(1,64)), 
                         ('action_net.bias', th.tensor([bias])),])},exact_match=False)
    env_eval = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=4*n_step-1)
        timesteps = timesteps + 4*n_step

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(-res_mean)
        res_std_arr.append(res_std)
        
        if -res_mean == min(res_mean_arr):
            cont_model.save("ppo_min_model_"+str(p)+"_"+str(L))
            
#     plt.plot(res_mean_arr,label="gae_lambda="+str(gae))
#     plt.xlabel("Iteration")
#     #plt.axhline(y=13.10673598, color='r', linestyle='-',label="optimal constant-order policy")
#     plt.ylabel("Average cost")
#     plt.title("L="+str(L)+", p="+str(p))
#     plt.show()
    
    min_model = PPO.load("ppo_min_model_"+str(p)+"_"+str(L))
    mean_min, std_min = ppo_evaluate(min_model, env_eval, 50000)
    print("p="+str(p)+"， L="+str(L)+": mean "+str(-mean_min)+", std_dev: "+str(std_min))
    
    return res_mean_arr, res_std_arr, mean_min, std_min

In [ ]:
#listp = [0.25,1,4,9,39,99]
#listL = [1,4,10,20,30,50,70,100]
#setting the parameters
p=99
L=1
n_iter = 10000
#gae lamda: suggest choose 0.95 when L<= 30, choose 0.99 when L>=50
gae_lambda = 0.95
learning_rate = 0.0003
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])
#bias suggest value: when p=0.25, bias=0.1; when p=1, bias=0.2; when p=4, bias=0.3; 
#when p=9, bias=0.4; when p=39, bias=0.5;when p=99, bias=0.6.
bias=0.6
# for p in listp:
#     for L in listL:

n_step = 2048 
t_t = 20*4*n_step
res_mean, res_std, mean_min, std_min = ppo_eval_interval(p,L,t_t, n_iter, n_step, gae_lambda, learning_rate,bias)
ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':-mean_min, 'res_std': std_min}, ignore_index=True)

In [ ]:
#codes for plotting the policy when L=1
min_model=PPO.load("results/ppo_min_model_"+str(1)+"_"+str(1))
a=range(0,50)
b=[0.1*i for i in a]
graph=np.zeros([50,50])
for i in range(50):
    for j in range(50):
        graph[i,j]=min_model.predict(([b[i]],[b[j]]),deterministic=True)[0]
X, Y = np.meshgrid(b, b)
plt.contourf(Y, X, graph, 9, cmap=plt.cm.viridis)
plt.colorbar()
plt.xlabel("On-hand Inventory")
plt.ylabel("Pipeline Inventory")
plt.title("PPO Policy Action for L=1, p=1")
print (graph)
